In [1]:
import pickle
from src.clf.constants import SVM_MODEL_MASKED
from sklearn import set_config

In [2]:
# get access to a folder outside of the src folder
import os
os.chdir('..')

In [3]:
# import model and display pipeline
set_config(display='diagram')
model = pickle.load(open(SVM_MODEL_MASKED, 'rb'))
model

Pipeline(steps=[('vect', CountVectorizer()), ('smote', SMOTE(random_state=42)),
                ('clf',
                 SVC(C=0.1, gamma=1, kernel='linear', probability=True,
                     random_state=42))])

In [6]:
# import mtsamples_finaloutput.csv as pandas dataframe
import pandas as pd
df = pd.read_csv('data/processed/nlp/mtsamples/mtsamples_finaloutput.csv')
# print full column width
pd.set_option('display.max_colwidth', None)
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,keywords_list,location,nr_candidates,top_n,keywords_outcome_weights_MLM,transcription_f_MLM,keywords_outcome_weights_TC,transcription_f_TC
0,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,mmode atrial enlargement atrial diameter normal size right ventricle normal lv systolic function ventricular ejection fraction normal lv diastolic function pericardial effusion normal morphology aortic valve mitral valve tricuspid valve pulmonary valve pa systolic pressure mmhgdoppler mild mitral tricuspid regurgitation trace aortic pulmonary regurgitation,"cardiovascular / pulmonary, 2-d m-mode, doppler, aortic valve, atrial enlargement, diastolic function, ejection fraction, mitral, mitral valve, pericardial effusion, pulmonary valve, regurgitation, systolic function, tricuspid, tricuspid valve, normal lv","['cardiovascular / pulmonary', ' 2-d m-mode', ' doppler', ' aortic valve', ' atrial enlargement', ' diastolic function', ' ejection fraction', ' mitral', ' mitral valve', ' pericardial effusion', ' pulmonary valve', ' regurgitation', ' systolic function', ' tricuspid', ' tricuspid valve', ' normal lv ']","dict_values([[221, 233], [11, 29], [163, 181], [135, 152], [234, 240], [234, 246], [182, 202], [263, 278], [334, 347], [99, 116], [247, 256], [247, 262], [89, 99]])",5,2,"[('tricuspid regurgitation', 0.882), ('pericardial effusion', 0.8454)]","['tricuspid regurgitation', 'pericardial effusion']","[('lv diastolic', 0.8428), ('effusion normal', 0.8311)]","['lv diastolic', 'effusion normal']"
1,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,ventricular cavity size wall thickness normal wall motion ventricular systolic function hyperdynamic estimate ejection fraction nearcavity obliteration increase ventricular outflow tract gradient mid cavity level consistent hyperdynamic ventricular systolic function abnormal ventricular relaxation pattern well elevate atrial pressure doppler atrium mildly dilate right atrium right ventricle normal aorti,"cardiovascular / pulmonary, 2-d, doppler, echocardiogram, annular, aortic root, aortic valve, atrial, atrium, calcification, cavity, ejection fraction, mitral, obliteration, outflow, regurgitation, relaxation pattern, stenosis, systolic function, tricuspid, valve, ventricular, ventricular cavity, wall motion, pulmonary artery","['cardiovascular / pulmonary', ' 2-d', ' doppler', ' echocardiogram', ' annular', ' aortic root', ' aortic valve', ' atrial', ' atrium', ' calcification', ' cavity', ' ejection fraction', ' mitral', ' obliteration', ' outflow', ' regurgitation', ' relaxation pattern', ' stenosis', ' systolic function', ' tricuspid', ' valve', ' ventricular', ' ventricular cavity', ' wall motion', ' pulmonary artery']","dict_values([[409, 416], [680, 687], [506, 517], [532, 544], [389, 395], [435, 441], [688, 701], [17, 23], [135, 152], [673, 679], [164, 176], [220, 227], [781, 794], [347, 365], [578, 586], [88, 105], [795, 804], [539, 544], [5, 16], [5, 23], [58, 69], [864, 880]])",5,2,"[('hyperdynamic ventricular', 0.8962), ('ventricular outflow', 0.8705)]","['hyperdynamic ventricular', 'ventricular outflow']","[('consistent hyperdynamic', 0.883), ('wall motion', 0.8099)]","['consistent hyperdynamic', 'wall motion']"
2,7,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 3,echocardiogrammultiple view heart great vessels normal intracardiac great vessel relationships cardiac function normal chamber enlargement hypertrophy pericardial effusion vegetations doppler interrogation color flow systemic venous return right atrium normal tricuspid inflow pulmonary outflow normal valve pulmonary venous return atrium interatrial septum intact mitral inflow ascend aorta flow normal aortic valve trileaflet coronary arteries normal or,"cardiovascular / pulmonary, 2-d echocardiogram, cardiac function, doppler, echocardiogram, multiple views, aortic valve, coronary arteri

In [7]:
# find if "abc" is in transcription_f_MLM
df['transcription_f_MLM'].str.contains('abc').sum()


18